In [1]:
#!pip install -r requirements.txt

In [2]:
import pandas as pd

In [15]:
#Exibir totas as colunas no pandas
pd.set_option('display.max_columns', None)

In [3]:
df_partidas = pd.read_parquet('data/raw/brasileirao_2023/partidas/')

In [18]:
df_partidas.head()

,fixture.id,fixture.referee,fixture.timezone,fixture.date,fixture.timestamp,fixture.periods.first,fixture.periods.second,fixture.venue.id,fixture.venue.name,fixture.venue.city,fixture.status.long,fixture.status.short,fixture.status.elapsed,league.id,league.name,league.country,league.logo,league.flag,league.season,league.round,teams.home.id,teams.home.name,teams.home.logo,teams.home.winner,teams.away.id,teams.away.name,teams.away.logo,teams.away.winner,goals.home,goals.away,score.halftime.home,score.halftime.away,score.fulltime.home,score.fulltime.away,score.extratime.home,score.extratime.away,score.penalty.home,score.penalty.away
0,1005649,"Rodrigo Jose Pereira de Lima, Brazil",America/Sao_Paulo,2023-04-16 16:00:00-03:00,1681671600,1.681672e+09,1.681675e+09,NaN,Estádio do Maracanã,Rio de Janeiro,Match Finished,FT,90.0,71,Serie A,Brazil,https://media-4.api-sports.io/football/leagues...,https://media-4.api-sports.io/flags/br.svg,2023,Regular Season - 1,127,Flamengo,https://media-4.api-sports.io/football/teams/1...,True,147,Coritiba,https://media-4.api-sports.io/football/teams/1...,False,3.0,0.0,1.0,0.0,3.0,0.0,None,None,None,None
1,1005650,"Ramon Abatti Abel, Brazil",America/Sao_Paulo,2023-04-15 18:30:00-03:00,1681594200,1.681594e+09,1.681598e+09,218.0,Estádio Nilton Santos,Rio de Janeiro,Match Finished,FT,90.0,71,Serie A,Brazil,https://media-4.api-sports.io/football/leagues...,https://media-4.api-sports.io/flags/br.svg,2023,Regular Season - 1,120,Botafogo,https://media-4.api-sports.io/football/teams/1...,True,126,Sao Paulo,https://media-4.api-sports.io/football/teams/1...,False,2.0,1.0,1.0,1.0,2.0,1.0,None,None,None,None
2,1005651,"Paulo Cesar Zanovelli da Silva, Brazil",America/Sao_Paulo,2023-04-15 16:00:00-03:00,1681585200,1.681585e+09,1.681589e+09,258.0,Allianz Parque,"São Paulo, São Paulo",Match Finished,FT,90.0,71,Serie A,Brazil,https://media-4.api-sports.io/football/leagues...,https://media-4.api-sports.io/flags/br.svg,2023,Regular Season - 1,121,Palmeiras,https://media-4.api-sports.io/football/teams/1...,True,1193,Cuiaba,https://media-4.api-sports.io/football/teams/1...,False,2.0,1.0,1.0,1.0,2.0,1.0,None,None,None,None
3,1005652,"Anderson Daronco, Brazil",America/Sao_Paulo,2023-04-16 16:00:00-03:00,1681671600,1.681672e+09,1.681675e+09,11531.0,Neo Química Arena,"São Paulo, São Paulo",Match Finished,FT,90.0,71,Serie A,Brazil,https://media-4.api-sports.io/football/leagues...,https://media-4.api-sports.io/flags/br.svg,2023,Regular Season - 1,131,Corinthians,https://media-4.api-sports.io/football/teams/1...,True,135,Cruzeiro,https://media-4.api-sports.io/football/teams/1...,False,2.0,1.0,0.0,0.0,2.0,1.0,None,None,None,None
4,1005653,"Maguielson Lima Barbosa, Brazil",America/Sao_Paulo,2023-04-15 18:30:00-03:00,1681594200,1.681594e+09,1.681598e+09,220.0,Estádio Nabi Abi Chedid,"Bragança Paulista, São Paulo",Match Finished,FT,90.0,71,Serie A,Brazil,https://media-4.api-sports.io/football/leagues...,https://media-4.api-sports.io/flags/br.svg,2023,Regular Season - 1,794,RB Bragantino,https://media-4.api-sports.io/football/teams/7...,True,118,Bahia,https://media-4.api-sports.io/football/teams/1...,False,2.0,1.0,0.0,1.0,2.0,1.0,None,None,None,None


In [16]:
df_partidas_stats = pd.read_parquet('data/raw/brasileirao_2023/partidas_estatistica/')

In [19]:
df_partidas_stats.head()

,type,value,team.id,team.name,fixture.id
0,Shots on Goal,7,127,Flamengo,1005649
1,Shots off Goal,5,127,Flamengo,1005649
2,Total Shots,15,127,Flamengo,1005649
3,Blocked Shots,3,127,Flamengo,1005649
4,Shots insidebox,5,127,Flamengo,1005649


In [13]:
df_wide = pd.pivot(df_partidas_stats, index=['fixture.id', 'team.name'], columns='type', values='value')

In [14]:
df_wide

type                           Ball Possession Blocked Shots Corner Kicks  \
fixture.id team.name                                                        
1005649    Coritiba                        41%             5            3   
           Flamengo                        59%             3            6   
1005650    Botafogo                        32%             4            3   
           Sao Paulo                       68%             2            7   
1005651    Cuiaba                          48%             2            4   
...                                        ...           ...          ...   
1005834    Bahia                           61%             5            9   
1005835    Fluminense                      71%             3            8   
           Gremio                          29%             2            3   
1005836    Atletico Paranaense             54%             2            5   
           Cuiaba                          46%             2            3   

type                           Fouls Goalkeeper Saves Offsides Passes %  \
fixture.id team.name                                                      
1005649    Coritiba               17                4        2      85%   
           Flamengo               20                3        1      89%   
1005650    Botafogo               20                7        2      78%   
           Sao Paulo              13                1        1      89%   
1005651    Cuiaba                 12                3        2      79%   
...                              ...              ...      ...      ...   
1005834    Bahia                  12                3        1      86%   
1005835    Fluminense              7                3        2      91%   
           Gremio                 11                6        3      76%   
1005836    Atletico Paranaense    22                1        1      81%   
           Cuiaba                 12                4        1      79%   

type                           Passes accurate Red Cards Shots insidebox  \
fixture.id team.name                                                       
1005649    Coritiba                        322      None               3   
           Flamengo                        480      None               5   
1005650    Botafogo                        229      None               6   
           Sao Paulo                       574      None              11   
1005651    Cuiaba                          337         1               6   
...                                        ...       ...             ...   
1005834    Bahia                           411         1               7   
1005835    Fluminense                      647      None               9   
           Gremio                          216      None               7   
1005836    Atletico Paranaense             346      None               8   
           Cuiaba                          286      None               3   

type                           Shots off Goal Shots on Goal Shots outsidebox  \
fixture.id team.name                                                           
1005649    Coritiba                         2             3                7   
           Flamengo                         5             7               10   
1005650    Botafogo                         5             3                6   
           Sao Paulo                        6             8                5   
1005651    Cuiaba                           6             4                6   
...                                       ...           ...              ...   
1005834    Bahia                            4             5                7   
1005835    Fluminense                       9             7               10   
           Gremio                           6             5                6   
1005836    Atletico Paranaense              4             6                4   
           Cuiaba                           8             1              

In [21]:
df_partidas[['fixture.id', 'teams.home.id', 'teams.home.name', 'teams.away.id', 'teams.away.name', 'teams.home.winner', 'teams.away.winner']]

,fixture.id,teams.home.id,teams.home.name,teams.away.id,teams.away.name,teams.home.winner,teams.away.winner
0,1005649,127,Flamengo,147,Coritiba,True,False
1,1005650,120,Botafogo,126,Sao Paulo,True,False
2,1005651,121,Palmeiras,1193,Cuiaba,True,False
3,1005652,131,Corinthians,135,Cruzeiro,True,False
4,1005653,794,RB Bragantino,118,Bahia,True,False
...,...,...,...,...,...,...,...
375,1006024,135,Cruzeiro,121,Palmeiras,None,None
376,1006025,119,Internacional,120,Botafogo,None,None
377,1006026,147,Coritiba,131,Corinthians,None,None
378,1006027,118,Bahia,1062,Atletico-MG,None,None
